You must run `neuroglancer_fileserver.ipynb` in parallel.

In [1]:
import sys
sys.path.insert(0, '..')

import neuroglancer as ng
import nibabel
import niizarr
import io
import base64
import numpy as np
import fsspec
import json
import os


In [2]:
HOME = os.environ.get('HOME')
PATH = f'{HOME}/Dropbox/data/niizarr'
PREFIX = 'http://127.0.0.1:8888/'
FNAME = 'sub-control01_T1w.nii.zarr'


In [3]:
with fsspec.open(os.path.join(PATH, FNAME, '.zattrs'), 'r') as f:
    zattrs = json.load(f)

# parse header with nibabel (we must fix magic string first)
hdr = niizarr.bin2nii(base64.b64decode(zattrs["nifti"]["base64"])).copy()
hdr["magic"] = "n+1"
hdr = nibabel.Nifti1Header.from_fileobj(io.BytesIO(hdr))

# compute matrices
permute = np.eye(4)
permute[:-1, :-1] = permute[:-1, [2, 1, 0]]
vox2ras = hdr.get_best_affine()
vox2phys = np.eye(4)
scale = zattrs["multiscales"][0]["datasets"][0]["coordinateTransformations"][0]["scale"]
shift = zattrs["multiscales"][0]["datasets"][0]["coordinateTransformations"][-1]["translation"]
vox2phys[[0, 1, 2], [0, 1, 2]] = list(reversed(scale[2:]))
vox2phys[:-1, -1] = list(reversed(shift[2:]))
phys2ras= vox2ras @ permute @ np.linalg.inv(vox2phys)

ras2lpi = np.eye(4)
ras2lpi[:-1, :-1] *= -1
phys2lpi = ras2lpi @ phys2ras

# define neuroglancer transform
ras_space = ng.CoordinateSpace(
    names=["x", "y", "z"],
    units="mm",
    scales=[1]*3,
)
lpi_space  = ng.CoordinateSpace(
    names=["left", "posterior", "inferior"],
    units="mm",
    scales=[1]*3,
)
phys_space = ng.CoordinateSpace(
    names=["z", "y", "x"],
    units="mm",
    scales=scale[2:],
)
transform = ng.CoordinateSpaceTransform(
    matrix=phys2lpi[:3, :4],
    input_dimensions=phys_space,
    output_dimensions=lpi_space,
)


In [4]:
# launch neuroglancer instance
ng.server.global_server_args['bind_port'] = '9090'
viewer = ng.Viewer(token='1')
print(viewer.get_viewer_url())


http://127.0.0.1:9090/v/1/


In [5]:
def make_freeview_layout():
    layout_col = {"type": "column", "children": []}
    for row in range(2):
        layout = {"type": "row", "children": []}
        for col in range(2):
            if row == col == 0:
                # axial
                layout["children"].append({"layout": "xy", "type": "viewer", "layers": ["mri"]})
            if row == 0 and col == 1:
                # sagittal
                layout["children"].append({"layout": "yz", "type": "viewer", "layers": ["mri"]})
            if row == col == 1:
                # coronal
                layout["children"].append({"layout": "xz", "type": "viewer", "layers": ["mri"]})
            if row == col == 1:
                # 3d view
                layout["children"].append({"layout": "3d", "type": "viewer", "layers": ["mri"]})
        layout_col["children"].append(layout)
    return layout_col

with viewer.txn() as state:
    state.layout = make_freeview_layout()


In [6]:
# load volume and transform
with viewer.txn() as state:
    state.layers.append(
        name="mri",
        layer=ng.ImageLayer(
                source=ng.LayerDataSource(
                url="zarr://" + os.path.join(PREFIX, FNAME),
                transform=transform,
            ),
            shader="""
            void main() {
                emitGrayscale(getDataValue()/1000.);
            }
            """
        )
    )

In [7]:
# for some reason if we don't wait, displayDimensions gets reset after
# I set them
import time
time.sleep(2)

with viewer.txn() as state:
    state.displayDimensions = ["left", "posterior", "inferior"]

In [8]:
# same, I need to force a state update after a while to avoid some of the
# FOV having large ugly voxels...
# Hopefully it's only a pyotebook thing
import time
time.sleep(2)


with viewer.txn() as state:
    viewer.shared_state.set_state(state)